In [1]:
%%configure
{ "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false",
            "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2"
          }}

In [2]:
import org.apache.spark.sql.{functions, SparkSession}
import org.apache.spark.sql.types.DataTypes
import org.apache.spark.sql.types.StructField
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.DataSourceReadOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.config.HoodieCompactionConfig
import org.apache.hudi.table.action.compact.CompactionTriggerStrategy
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.streaming.StreamingQueryListener._
import org.apache.spark.sql.{DataFrame, Row, SaveMode}
import java.time.LocalDateTime

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1647862247100_0009,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.{functions, SparkSession}
import org.apache.spark.sql.types.DataTypes
import org.apache.spark.sql.types.StructField
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.DataSourceReadOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.config.HoodieCompactionConfig
import org.apache.hudi.table.action.compact.CompactionTriggerStrategy
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.streaming.StreamingQueryListener._
import org.apache.spark.sql.{DataFrame, Row, SaveMode}
import java.time.LocalDateTime


In [3]:
// Spark Session - Add listeners
val spark = SparkSession.builder.appName("SparkHudi").
                config("spark.serializer", "org.apache.spark.serializer.KryoSerializer").
                config("spark.default.parallelism", 9).
                config("spark.sql.shuffle.partitions", 9).
                enableHiveSupport().
                getOrCreate()

spark.streams.addListener(new StreamingQueryListener() {
    override def onQueryStarted(queryStarted: QueryStartedEvent): Unit = {
        println("Query started: " + queryStarted.id)
    }
    override def onQueryTerminated(queryTerminated: QueryTerminatedEvent): Unit = {
        println("Query terminated: " + queryTerminated.id)
    }
    override def onQueryProgress(queryProgress: QueryProgressEvent): Unit = {
        println("Query made progress: " + queryProgress.progress)
    }
})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@39341a8


In [4]:
// Define kafka flow
val targetDir = "/tmp/sparkHudi"
val checkpointLocation = "/tmp/sparkHudi/checkpoint/"
val bootstrapServers = "b-1.corrine-kafka.9wq81s.c5.kafka.us-east-1.amazonaws.com:9092,b-3.corrine-kafka.9wq81s.c5.kafka.us-east-1.amazonaws.com:9092,b-2.corrine-kafka.9wq81s.c5.kafka.us-east-1.amazonaws.com:9092"
val topic = "CorrineNewTopic"

val schema = DataTypes.createStructType(Array[StructField](
  DataTypes.createStructField("id", DataTypes.LongType, false),
  DataTypes.createStructField("date", DataTypes.DateType, false),
  DataTypes.createStructField("name", DataTypes.StringType, false),
  DataTypes.createStructField("sales", DataTypes.StringType, false)))

val dataStreamReader = spark.readStream.format("kafka").
                    option("kafka.bootstrap.servers", bootstrapServers).
                    option("subscribe", topic).
                    option("maxOffsetsPerTrigger", 1000).
                    option("startingOffsets", "earliest").
                    option("failOnDataLoss", value = false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

targetDir: String = /tmp/sparkHudi
checkpointLocation: String = /tmp/sparkHudi/checkpoint/
bootstrapServers: String = b-1.corrine-kafka.9wq81s.c5.kafka.us-east-1.amazonaws.com:9092,b-3.corrine-kafka.9wq81s.c5.kafka.us-east-1.amazonaws.com:9092,b-2.corrine-kafka.9wq81s.c5.kafka.us-east-1.amazonaws.com:9092
topic: String = CorrineNewTopic
schema: org.apache.spark.sql.types.StructType = StructType(StructField(id,LongType,false), StructField(date,DateType,false), StructField(name,StringType,false), StructField(sales,StringType,false))
dataStreamReader: org.apache.spark.sql.streaming.DataStreamReader = org.apache.spark.sql.streaming.DataStreamReader@a6f980a


In [5]:
// Loading stream data
val df = dataStreamReader.load().selectExpr(
        "topic as kafka_topic",
        "CAST(partition AS STRING) kafka_partition",
        "CAST(timestamp AS String) kafka_timestamp",
        "CAST(offset AS STRING) kafka_offset",
        "CAST(key AS STRING) kafka_key",
        "CAST(value AS STRING) kafka_value",
        "current_timestamp() current_time",
).selectExpr(
        "kafka_topic",
 		"concat(kafka_partition,'-',kafka_offset) kafka_partition_offset",
        "kafka_offset",
        "kafka_timestamp",
        "kafka_key",
        "kafka_value",
    	"substr(current_time,1,10) partition_date")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [kafka_topic: string, kafka_partition_offset: string ... 5 more fields]


In [6]:
val new_df = df.select(from_json(col("kafka_value"), schema).as("value")).select("value.*")
new_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

new_df: org.apache.spark.sql.DataFrame = [id: bigint, date: date ... 2 more fields]
root
 |-- id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- sales: string (nullable = true)



In [7]:
// Hudi Options
val s3Bucket = "s3://emr-hudi-corrine"
val TABLE = "sales"
val RECORDKEY = "id"
val PARTITION_FIELDS = "name"
val PRECOMBINE_FIELD = "date"
// Skip the last five configs from your original hudi options
val hudiOptions = Map[String,String](
  HoodieWriteConfig.TABLE_NAME -> TABLE,
  DataSourceWriteOptions.TABLE_TYPE_OPT_KEY -> DataSourceWriteOptions.MOR_TABLE_TYPE_OPT_VAL,
  DataSourceWriteOptions.RECORDKEY_FIELD_OPT_KEY -> RECORDKEY,
  DataSourceWriteOptions.PARTITIONPATH_FIELD_OPT_KEY -> PARTITION_FIELDS,
  DataSourceWriteOptions.PRECOMBINE_FIELD_OPT_KEY -> PRECOMBINE_FIELD,
  DataSourceWriteOptions.HIVE_SYNC_ENABLED_OPT_KEY -> "false",
  HoodieCompactionConfig.INLINE_COMPACT_TRIGGER_STRATEGY_PROP -> CompactionTriggerStrategy.TIME_ELAPSED.name,
  HoodieCompactionConfig.INLINE_COMPACT_TIME_DELTA_SECONDS_PROP -> String.valueOf(60 * 60),
  HoodieCompactionConfig.CLEANER_COMMITS_RETAINED_PROP -> "168",
  HoodieCompactionConfig.ASYNC_CLEAN_PROP -> "false", 
  HoodieCompactionConfig.INLINE_COMPACT_PROP -> "true",
  HoodieCompactionConfig.MIN_COMMITS_TO_KEEP_PROP -> "169",
  HoodieCompactionConfig.MAX_COMMITS_TO_KEEP_PROP -> "300"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3Bucket: String = s3://emr-hudi-corrine
TABLE: String = sales
RECORDKEY: String = id
PARTITION_FIELDS: String = name
PRECOMBINE_FIELD: String = date
hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.compact.inline.max.delta.seconds -> 3600, hoodie.datasource.write.precombine.field -> date, hoodie.compact.inline.trigger.strategy -> TIME_ELAPSED, hoodie.datasource.hive_sync.enable -> false, hoodie.datasource.write.recordkey.field -> id, hoodie.table.name -> sales, hoodie.datasource.write.table.type -> MERGE_ON_READ, hoodie.compact.inline -> true, hoodie.keep.max.commits -> 300, hoodie.cleaner.commits.retained -> 168, hoodie.keep.min.commits -> 169, hoodie.datasource.write.partitionpath.field -> name, hoodie.clean.async -> false)


In [ ]:
// Create and start query
def myFunc(batchDF: DataFrame, batchId: Long) : Unit = {
        batchDF.persist()
        println(LocalDateTime.now() + "start writing mor table")
		batchDF.write.format("org.apache.hudi").
            option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.INSERT_OPERATION_OPT_VAL).
            options(hudiOptions).
            mode(SaveMode.Append).
            save(s"${s3Bucket}/${TABLE}")
        println(LocalDateTime.now() + "finish")
        batchDF.unpersist()
}
val query = new_df.writeStream.queryName("streaming_hudi_query").
      foreachBatch(myFunc _).
      start()

query.awaitTermination()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…